In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://127.0.0.1:9200')

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [11]:
from tqdm.auto import tqdm

In [12]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [11]:
query = 'How do I execute a command in a running docker container?'

In [12]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [13]:
response = es_client.search(index=index_name, body=search_query)

In [14]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [18]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text'])
        context = context + "\n\n"

    context = context.strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [19]:
prompt = build_prompt(query, result_docs)

In [20]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [21]:
len(prompt)

1462

In [22]:
import tiktoken

In [23]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [25]:
len(encoding.encode(prompt))

322

In [26]:
from openai import OpenAI

In [27]:
client = OpenAI()

In [28]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

In [30]:
print(response.choices[0].message.content)

To execute a command in a running Docker container, follow these steps:

1. Use `docker ps` to find the container ID of the running container.
   ```sh
   docker ps
   ```
2. Use `docker exec` to run a command inside the container. For example, to start a bash session:
   ```sh
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual container ID obtained from the `docker ps` command.


In [34]:
response.dict()['usage']

{'completion_tokens': 102, 'prompt_tokens': 329, 'total_tokens': 431}

In [36]:
input_cost = (response.dict()['usage']['prompt_tokens']/1000000)*5
output_cost = (response.dict()['usage']['completion_tokens']/1000000)*15
total_cost = input_cost + output_cost

In [38]:
print(input_cost, output_cost, total_cost)

0.0016449999999999998 0.00153 0.003175
